In [1]:
# YOLOv11 Training Script with Full Parameter Explanations and Improved Stability

from ultralytics import YOLO
import os
from datetime import datetime

# --- Model Setup ---
# Load a pretrained YOLOv8n model (nano version: fastest, smallest, good for limited data or edge devices)
model = YOLO('../src/human_detector/models/yolov11n.pt')
# To resume training from a previous checkpoint:
# model = YOLO('/path/to/your/best.pt')

# --- Training Configuration ---
epochs = 300               # Number of times the model sees the entire dataset
imgsz = 640                # Size to which all images are resized (square). 640 is a good trade-off between speed and accuracy
batch_size = 4             # Number of images used in one forward/backward pass (small for limited VRAM)

lr0 = 1e-5                 # Initial learning rate (controls how much the model updates each step)
lrf = 0.01                 # Final learning rate = lr0 * lrf, used with cosine decay learning schedule
optimizer = "Adam"          # Optimizer algorithm: 'SGD' is more stable for small datasets, 'AdamW' (default) adapts better to large, noisy datasets
# warmup_epochs = 3          # Number of epochs to slowly ramp up learning rate to avoid instability at start
dropout = 0.1              # Dropout rate (randomly turns off neurons during training to prevent overfitting)

# --- Experiment Folder Naming ---
time_stamp = datetime.now().strftime("%Y%m%d_%H%M")

exp_name = (
    f"yolov11n_v2_rect_false_"         # Model name and version. You can use this to identify custom model variants.
    f"ep{epochs}_"          # Number of epochs (e.g., ep300 means trained for 300 full dataset passes)
    f"img{imgsz}_"          # Image input size (e.g., img640 means all images resized to 640x640)
    f"bs{batch_size}_"      # Batch size (e.g., bs4 means 4 images processed per training step)
    f"lr{lr0}_"             # Initial learning rate (e.g., lr0.0001)
    f"lrf{lrf}_"            # Final learning rate factor (e.g., lrf0.01 → lr drops to 0.000001 at end)
    f"{optimizer}_"         # Optimizer used (e.g., SGD, AdamW, etc.)
    f"augmented_"            # Tag for augmentation setup (e.g., Final or custom profile name)
    f"{time_stamp}"         # Timestamp for uniqueness and chronological sorting
)

# --- Training Start ---
results = model.train(
    seed=42,                     # Random seed for reproducibility
    data='data.yaml',            # Path to data config file (defines class names and image paths)
    epochs=epochs,               # Total number of training epochs
    patience=50,                 # Early stopping: if val loss doesn't improve for 10 epochs, stop training
    dropout=dropout,             # Helps prevent overfitting by randomly disabling neurons
    imgsz=imgsz,                 # Resizes all images to this square dimension
    batch=batch_size,            # Number of images per training batch
    lr0=lr0,                     # Starting learning rate
    lrf=lrf,                     # Final learning rate multiplier
    # warmup_epochs=warmup_epochs,# Smooth ramp-up of LR to avoid unstable gradients
    optimizer=optimizer,         # Optimizer choice: SGD recommended for small datasets
    device=0,                    # GPU device ID (0 = first GPU)
    half=True,                   # Use float16 precision if supported (faster training)
    workers=2,                   # Number of CPU workers for data loading (adjust based on CPU)
    cache=True,                  # Preload and cache images in memory for faster epoch execution
    # rect=True,                   # Enables rectangular training for better image aspect ratio handling
    verbose=True,                # Print detailed training logs for every batch
    amp=False ,
    # exist_ok=True,               # Overwrites existing output folder if it already exists

    # --- Data Augmentation (tuned for small datasets) ---
    hsv_h=0.015,                 # Hue variation (color tint)
    hsv_s=0.7,                   # Saturation variation (color strength)
    hsv_v=0.4,                   # Brightness variation
    translate=0.05,              # Random image shift (reduced for safety)
    scale=0.1,                   # Random zoom in/out (less aggressive)
    fliplr=0.5,                  # 50% chance to flip images horizontally
    mosaic=0.2,                  # Combines 4 training images into one
    mixup=0.03,                  # Blends two images and labels
    copy_paste=0.03,             # Copies object from one image and pastes onto another
    perspective=0.001,           # Small perspective distortion to improve robustness

    # --- Output Settings ---
    project='results',           # Base directory to store training logs and weights
    name=exp_name                # Subfolder name under project/ to store this run’s output
)

# --- Save Path Display ---
save_dir = str(results.save_dir)
best_pt = os.path.join(save_dir, 'weights', 'best.pt')
print(f"✅ Trained checkpoint saved to: {best_pt}")


New https://pypi.org/project/ultralytics/8.3.150 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.149 🚀 Python-3.10.15 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7933MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.03, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.03, mode=train, model=../src/human_detector/models/yolov11n.pt, momentum=0.937, mosaic=0.2, multi_scale=False, name=yolov11n_v2_rect

100%|██████████| 755k/755k [00:00<00:00, 28.1MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,230 parameters, 2,590,214 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 962.1±61.9 MB/s, size: 349.1 KB)


train: Scanning /home/commu/Desktop/human_detector_ws/data_training/data_teleco/Video_1/labels/train... 2575 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2575/2575 [00:12<00:00, 205.75it/s]

...

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.6GB RAM): 100%|██████████| 644/644 [00:00<00:00, 1279.53it/s]


Plotting labels to results/yolov11n_v2_rect_false_ep300_img640_bs4_lr1e-05_lrf0.01_Adam_augmented_20250605_1231/labels.jpg... 
optimizer: Adam(lr=1e-05, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to results/yolov11n_v2_rect_false_ep300_img640_bs4_lr1e-05_lrf0.01_Adam_augmented_20250605_1231
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      1.25G       1.17     0.9702      1.296         19        640:  62%|██████▏   | 399/644 [00:21<00:13, 18.73it/s]


KeyboardInterrupt: 

# 🧠 YOLOv8 Training Metrics Cheatsheet

## 🔢 Loss Curves

| Metric            | Meaning                                                | Goal / Good Trend                |
|-------------------|--------------------------------------------------------|----------------------------------|
| `train/box_loss`  | Bounding box regression loss (training)                | ↓ Steady decline                 |
| `train/cls_loss`  | Classification loss (training)                         | ↓ Should fall and stabilize     |
| `train/dfl_loss`  | Distribution Focal Loss (for bounding box quality)     | ↓ Smooth drop                   |
| `val/box_loss`    | Box loss on validation set                             | ↓ Best indicator of generalization |
| `val/cls_loss`    | Classification loss on validation                      | ↓ Falling → Good model fit      |
| `val/dfl_loss`    | DFL loss on validation                                 | ↓ Matches train trend = no overfit |

---

## 📈 Detection Metrics

| Metric                | Description                                                 | Goal / Interpretation           |
|------------------------|-------------------------------------------------------------|----------------------------------|
| `metrics/precision(B)` | % of correct detections out of all predictions              | ↑ High = few false positives     |
| `metrics/recall(B)`    | % of actual objects detected                                | ↑ High = few missed detections   |
| `metrics/mAP50(B)`     | Accuracy of boxes at 50% IoU threshold                     | ↑ Main object detection metric   |
| `metrics/mAP50-95(B)`  | Average mAP over IoU 0.5 to 0.95 (stricter)                | ↑ Robust performance measure     |

---

## 🧪 Ideal Curve Patterns

- **Losses**: should steadily decrease.
- **Validation loss** rising while training loss decreases → possible **overfitting**.
- **Precision & Recall**: should increase and plateau near 1.0.
- **mAP**: aim for **>0.90 mAP50** and **>0.80 mAP50-95** for high-quality results.

---

## ⚠️ Tips

- 🧊 Use `half=True` for mixed precision training on supported GPUs.
- 🧠 Watch `val/box_loss` and `mAP50-95` to catch overfitting early.
- 🧪 If metrics plateau early, try increasing `epochs`, `imgsz`, or improving labels.
- 💾 Check `runs/train/.../results.png` for curve visualization.

---
